# Classification Experiment on AMI-Br (Patch-Level Split)

In this experiment, we split up the data on patch level, i.e., we do not make sure to not have patches of the same patient in train/validation and test set. The reason we don't do this is that this experiment aims to only demonstrate label consistency.

The pipeline uses only basic augmentation and trials two architectures:
- EfficientNetV2-S
- DenseNet121
  
and three strategies:
- weighted sampling (similar to minority oversampling with augmentation)
- weighted loss (cross-entropy)
- focal loss


In [1]:
## import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data import Subset
from torchvision.ops import sigmoid_focal_loss


from dataset import AMFDataset

import albumentations as A
import numpy as np
from tiatoolbox.tools.stainaugment import StainAugmentor
from albumentations.pytorch import ToTensorV2

stain_matrix = np.array([[0.91633014, -0.20408072, -0.34451435],
               [0.17669817, 0.92528011, 0.33561059]])

aug_pipline = A.Compose([
                        A.RandomRotate90(),
                        A.HorizontalFlip(0.5),
                        A.VerticalFlip(0.5),
                        StainAugmentor(method='macenko', stain_matrix=stain_matrix),
                        A.RandomBrightnessContrast(p=0.2),
                        ToTensorV2(),
                        ])

nonaug_pipeline = A.Compose([
    ToTensorV2(),  # Convert to tensor
])


/home/aubreville/venv/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# import the library
import wandb


In [3]:
import json
from torch import nn
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from torch.utils.data import WeightedRandomSampler
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import random_split

class EfficientNetClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(EfficientNetClassifier, self).__init__()
        # Load pre-trained EfficientNet_V2_S model
        self.efficientnet = models.efficientnet_v2_s(weights=models.efficientnet.EfficientNet_V2_S_Weights.DEFAULT)
        # Replace the classifier
        self.efficientnet.classifier = nn.Linear(list(self.efficientnet.classifier.modules())[-1].in_features, num_classes)

    def forward(self, x):
        return self.efficientnet(x)

class DenseNetClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(DenseNetClassifier, self).__init__()
        # Load pre-trained DenseNet model
        self.densenet = models.densenet121(weights=models.densenet.DenseNet121_Weights.DEFAULT)
        # Replace the classifier
        self.densenet.classifier = nn.Linear(self.densenet.classifier.in_features, num_classes)

    def forward(self, x):
        return self.densenet(x)

# retrieved from: https://github.com/clcarwin/focal_loss_pytorch/blob/master/focalloss.py
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()
            


def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, condition, num_epochs=10, device='cuda'):
    model.to(device)
    best_acc = 0

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0
        val_running_corrects = 0
        val_total_samples = 0
        all_labels = []
        all_preds = []
        all_probs = []  # To store predicted probabilities for AUC-ROC

        
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.float()
            labels = labels.to(device).long()
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Backward pass + optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total_samples += labels.size(0)

        
        for inputs, labels in val_loader:
            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = inputs.float()
                labels = labels.to(device).long()
                
                
                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                probs = torch.softmax(outputs, dim=1)[:, 1]
                
                val_running_corrects += torch.sum(preds == labels.data)
                val_total_samples += labels.size(0)

                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())
                all_probs.extend(probs.cpu().detach().numpy())  # Detach probabilities for AUC-ROC

        balanced_acc = balanced_accuracy_score(all_labels, all_preds)
        auc_roc = roc_auc_score(all_labels, all_probs)
        if balanced_acc>best_acc:
            torch.save(model.state_dict(), f'best_model_ckpt.pt')
            print('New best model found.')
            best_acc=balanced_acc

        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects.to(torch.device('cpu')).double() / total_samples
        val_epoch_acc = val_running_corrects.to(torch.device('cpu')).double() / val_total_samples
        wandb.log({'training_loss': running_loss,
                   'validation_acc' : val_epoch_acc,
                   'validation_balanced_acc': balanced_acc,
                   'validation_auc' : auc_roc})
        
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, Val Accuracy {val_epoch_acc:.4f}, balanced_acc: {balanced_acc:.4f}, roc_auc_score: {auc_roc:.4f} ')
    
    all_labels = []
    all_preds = []
    all_probs = []  # To store predicted probabilities for AUC-ROC

    model.load_state_dict(torch.load('best_model_ckpt.pt', weights_only=True))
    for inputs, labels in test_loader:
        with torch.no_grad():
            inputs, labels = inputs.to(device).float(), labels.to(device).long()
            

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            
            val_running_corrects += torch.sum(preds == labels.data)
            val_total_samples += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().detach().numpy())  # Detach probabilities for AUC-ROC    
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    auc_roc = roc_auc_score(all_labels, all_probs)
    log = { 'run' : epoch,
            'val_max_balanced_acc' : best_acc,
            'test_balanced_acc': balanced_acc,
            'test_auc' : auc_roc}
    try:
        wandb.log(log)
    except:
        pass

    print(f'TEST: balanced_acc: {balanced_acc:.4f}, roc_auc_score: {auc_roc:.4f}')
    f = open(f'classification_results_nopatientsplit_{condition}.log','a')
    f.write(json.dumps(log)+'\n')
    f.close()
    
    return model, all_probs, all_labels, all_preds

import pandas as pd
from sklearn.model_selection import train_test_split

def main(num_epochs=30, arch='EfficientNetV2', strategy='weightedsampling', learning_rate=0.001, class_weights=None):
    amibr_df = pd.read_csv('AMI-BR.csv')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Train the model
    for k in range(5):
        run = wandb.init(
            project="AMI-Br", 
            name=f"run_{k}_nopatientsplit_{arch}_{strategy}", 
            id=f"run_{k}_nopatientsplit_{arch}_{strategy}",
        )            
        print('5-fold-MC-CV, Run: ',k)

        dataset = AMFDataset(root_dir='patches', transform=aug_pipline, ids=None)
        all_labels = [y for x,y in dataset.images]

        dataset_train, valtest_dataset = random_split(dataset, [0.7, 0.3])
        dataset_valid, dataset_test = random_split(valtest_dataset, [0.5, 0.5])

        print('TRAINING OVERVIEW:\n')
        print('==================\n')

        print('Number of cells in train: ',len(dataset_train),'(%.2f percent)' % (100*len(dataset_train)/len(amibr_df)))
        print('Number of cells in valid: ',len(dataset_valid),'(%.2f percent)' % (100*len(dataset_valid)/len(amibr_df)))
        print('Number of cells in test: ',len(dataset_test),'(%.2f percent)' % (100*len(dataset_test)/len(amibr_df)))


        # Sampler for minority class undersampling
        
        train_labels = np.array(all_labels)[dataset_train.indices].tolist()
        count_amf = train_labels.count('atypical')
        count_nmf = train_labels.count('normal')
        weights = [1/count_amf if (x=='atypical') else 1/count_nmf for x in train_labels]

        sampler = WeightedRandomSampler(weights=weights, num_samples=len(train_labels), replacement=True)

        # Create dataloader
        if (strategy=='weightedsampling'):
            dataloader_train = DataLoader(dataset_train, batch_size=32, num_workers=8, sampler=sampler)
            criterion = nn.CrossEntropyLoss()
        elif (strategy=='weightedloss'):
            dataloader_train = DataLoader(dataset_train, batch_size=32, num_workers=8, shuffle=True)
            class_weights = [0.776, 0.224]  # Weights to reflect the dataset distribution
            weights = torch.tensor(class_weights, dtype=torch.float).to(device)
            criterion = nn.CrossEntropyLoss(weight=weights)
        elif (strategy=='focalloss'):
            dataloader_train = DataLoader(dataset_train, batch_size=32, num_workers=8, shuffle=True)
            criterion = FocalLoss()

        dataloader_valid = DataLoader(dataset_valid, batch_size=32, shuffle=False)
        dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

        # Initialize model, loss, optimizer
        if (arch == 'EfficientNetV2'):
            model = EfficientNetClassifier(num_classes=2).to(device)
        elif (arch == 'DenseNet121'):
            model = DenseNetClassifier(num_classes=2).to(device)
        else:
            raise ValueError('please specify valid architecture')

        
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

        trained_model,all_probs, all_labels, all_preds = train_model(model, dataloader_train, dataloader_valid, dataloader_test, criterion, optimizer, condition=arch+'_'+strategy, num_epochs=num_epochs, device=device)
        run.finish()
    return trained_model,all_probs, all_labels, all_preds



In [4]:
trained_model,all_probs, all_labels, all_preds = main(num_epochs=100, arch='DenseNet121', strategy='weightedsampling', learning_rate=0.001)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: maubreville (muromi). Use `wandb login --relogin` to force relogin


5-fold-MC-CV, Run:  0
['atypical', 'normal']
TRAINING OVERVIEW:


Number of cells in train:  2604 (70.00 percent)
Number of cells in valid:  558 (15.00 percent)
Number of cells in test:  558 (15.00 percent)



00%|███████████████████████████████████████████| 82/82 [00:10<00:00,  8.19it/s]

New best model found.
Epoch 0/99, Loss: 0.6444, Accuracy: 0.6448, Val Accuracy 0.6756, balanced_acc: 0.7068, roc_auc_score: 0.7681 



00%|███████████████████████████████████████████| 82/82 [00:11<00:00,  6.90it/s]

...


00%|███████████████████████████████████████████| 82/82 [00:12<00:00,  6.53it/s]

Epoch 98/99, Loss: 0.0597, Accuracy: 0.9773, Val Accuracy 0.7975, balanced_acc: 0.8025, roc_auc_score: 0.8713 



00%|███████████████████████████████████████████| 82/82 [00:12<00:00,  6.58it/s]

Epoch 99/99, Loss: 0.0627, Accuracy: 0.9781, Val Accuracy 0.8226, balanced_acc: 0.8156, roc_auc_score: 0.8750 
TEST: balanced_acc: 0.7814, roc_auc_score: 0.8765


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▇▆▆▆▆▅▅▅▄▄▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁
val_max_balanced_acc,▁
validation_acc,▁▅▄▄▄▄▃▃▃▆▆▄▆▆▅▆▆█▇▇▇▆▇▆▇█▇▇▇▆▆▇▆█▇▇▇▇█▇
validation_auc,▁▂▄▅▇▅▅▇▆▇▇▇█▇▆▇▇▇█▇█▇▇▇█▇▇▆▆▇▇▆▇▇▆▇▇▇▇▆
validation_balanced_acc,▁▄▆▅▅▆▅▅▅▇▆▇▇▆▆▆█▇▇▇▇▆▇▆▆▆█▆▇▇▅▇▇▇▇▆▇▆▇▇
run,99
test_auc,0.87648
test_balanced_acc,0.78138


In [5]:
trained_model,all_probs, all_labels, all_preds = main(num_epochs=100, arch='EfficientNetV2', strategy='weightedsampling', learning_rate=0.001)


5-fold-MC-CV, Run:  0
['atypical', 'normal']
TRAINING OVERVIEW:


Number of cells in train:  2604 (70.00 percent)
Number of cells in valid:  558 (15.00 percent)
Number of cells in test:  558 (15.00 percent)



00%|███████████████████████████████████████████| 82/82 [00:11<00:00,  7.27it/s]

New best model found.
Epoch 0/99, Loss: 0.6340, Accuracy: 0.6306, Val Accuracy 0.6487, balanced_acc: 0.6851, roc_auc_score: 0.7642 



00%|███████████████████████████████████████████| 82/82 [00:12<00:00,  6.34it/s]

New best model found.
Epoch 1/99, Loss: 0.5115, Accuracy: 0.7611, Val Accuracy 0.7401, balanced_acc: 0.7378, roc_auc_score: 0.7921 


...


00%|███████████████████████████████████████████| 82/82 [00:12<00:00,  6.45it/s]

Epoch 98/99, Loss: 0.0524, Accuracy: 0.9831, Val Accuracy 0.7957, balanced_acc: 0.8004, roc_auc_score: 0.8636 



00%|███████████████████████████████████████████| 82/82 [00:13<00:00,  6.24it/s]

Epoch 99/99, Loss: 0.0477, Accuracy: 0.9789, Val Accuracy 0.8100, balanced_acc: 0.8044, roc_auc_score: 0.8707 
TEST: balanced_acc: 0.7969, roc_auc_score: 0.8887


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▆▆▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_max_balanced_acc,▁
validation_acc,▂▃▄▂▁▄▁▄▄▄▄▄▄▅▆▆▆▆▅▇▆▆▇▅▆▆▇▅▇▇▆▇▅▅▆█▇▆▇▆
validation_auc,▁▂▂▂▄▅▄▄▅▄▅▆▅▅▆▄▅▅▇▆▇▅▆▆▄▅▅▄▄▅▆▆▆▆▅█▇▆▅▆
validation_balanced_acc,▁▂▅▄▆▅▆█▇▅█▅▅▇▆▆▅▇▆▇▆█▅▅▇▆▇▆▇▆▄▄▄▇▆█▇▄██
run,99
test_auc,0.88872
test_balanced_acc,0.79692


In [6]:
trained_model = main(num_epochs=100, arch='DenseNet121', strategy='weightedloss', learning_rate=0.001)


5-fold-MC-CV, Run:  0
['atypical', 'normal']
TRAINING OVERVIEW:


Number of cells in train:  2604 (70.00 percent)
Number of cells in valid:  558 (15.00 percent)
Number of cells in test:  558 (15.00 percent)



00%|███████████████████████████████████████████| 82/82 [00:08<00:00,  9.19it/s]

New best model found.
Epoch 0/99, Loss: 0.7654, Accuracy: 0.5776, Val Accuracy 0.7401, balanced_acc: 0.5491, roc_auc_score: 0.6129 



00%|███████████████████████████████████████████| 82/82 [00:10<00:00,  7.57it/s]

New best model found.
Epoch 1/99, Loss: 0.6709, Accuracy: 0.6974, Val Accuracy 0.7957, balanced_acc: 0.7428, roc_auc_score: 0.8042 


...


00%|███████████████████████████████████████████| 82/82 [00:13<00:00,  6.19it/s]

Epoch 98/99, Loss: 0.1196, Accuracy: 0.9516, Val Accuracy 0.7903, balanced_acc: 0.7348, roc_auc_score: 0.8492 



00%|███████████████████████████████████████████| 82/82 [00:12<00:00,  6.80it/s]

Epoch 99/99, Loss: 0.1249, Accuracy: 0.9435, Val Accuracy 0.8136, balanced_acc: 0.7635, roc_auc_score: 0.8520 
TEST: balanced_acc: 0.8232, roc_auc_score: 0.8868


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▅▅▅▅▄▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_max_balanced_acc,▁
validation_acc,▁▃▅▄▃▆▅▆▇▆▆▆▅▇▇▇▇▆▇▇▇▇▆▇▇█▇▇██▆▇▇▇██▇▇█▇
validation_auc,▁▂▆▅▆▇▆▇▆▆▇▇▇▇█▇▇▆▇▇▇▆▇▇▇▇▇█▇▇▇▇▇▇▇▇█▇▇▇
validation_balanced_acc,▁▁▂▂▃▃▆▅▄▄▁▄▆▇▆▇▆█▆▅▆▇▆▄▃▇▅▅▇▅▄▃▆▆▃█▆▃▆▃
run,99
test_auc,0.88677
test_balanced_acc,0.82319


In [7]:

trained_model = main(num_epochs=100, arch='EfficientNetV2', strategy='weightedloss', learning_rate=0.001)


5-fold-MC-CV, Run:  0
['atypical', 'normal']
TRAINING OVERVIEW:


Number of cells in train:  2604 (70.00 percent)
Number of cells in valid:  558 (15.00 percent)
Number of cells in test:  558 (15.00 percent)



00%|███████████████████████████████████████████| 82/82 [00:10<00:00,  7.69it/s]

New best model found.
Epoch 0/99, Loss: 0.6451, Accuracy: 0.6528, Val Accuracy 0.7760, balanced_acc: 0.7445, roc_auc_score: 0.8221 



00%|███████████████████████████████████████████| 82/82 [00:13<00:00,  6.16it/s]

New best model found.
Epoch 1/99, Loss: 0.5298, Accuracy: 0.7680, Val Accuracy 0.7616, balanced_acc: 0.7560, roc_auc_score: 0.8232 


...


00%|███████████████████████████████████████████| 82/82 [00:10<00:00,  7.90it/s]

Epoch 98/99, Loss: 0.0716, Accuracy: 0.9712, Val Accuracy 0.8315, balanced_acc: 0.7335, roc_auc_score: 0.8493 



00%|███████████████████████████████████████████| 82/82 [00:13<00:00,  6.24it/s]

Epoch 99/99, Loss: 0.0711, Accuracy: 0.9716, Val Accuracy 0.8405, balanced_acc: 0.7674, roc_auc_score: 0.8750 
TEST: balanced_acc: 0.7842, roc_auc_score: 0.8769


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_max_balanced_acc,▁
validation_acc,▁▅▅▂▃▆▇▃▅▃▅▅▄▇▆█▆█▆▇▆▄▆▄▇▇▃▆▇▆▅▇▇▇▇▆▆▆█▇
validation_auc,▂▁▆█▆▅▅▅▇▇▇▇▆▅▆▇▇▆▆▇▇▅▆█▇▆▆▇▆▅▅▆▅▇▇▇▆▇▆▇
validation_balanced_acc,▁▅▆▄▅▄▃▃▄█▆▅▆▆▅▃█▆▆█▇▄▆▄▃▆▇▇▇▄▅▆▅▇▄▇▂▆▇▂
run,99
test_auc,0.8769
test_balanced_acc,0.78419


In [8]:
trained_model = main(num_epochs=100, arch='DenseNet121', strategy='focalloss', learning_rate=0.001)


5-fold-MC-CV, Run:  0
['atypical', 'normal']
TRAINING OVERVIEW:


Number of cells in train:  2604 (70.00 percent)
Number of cells in valid:  558 (15.00 percent)
Number of cells in test:  558 (15.00 percent)


|2025-01-03|20:10:48.093| [WARNING] /tmp/ipykernel_873295/3560450480.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 82/82 [00:10<00:00,  8.13it/s]

New best model found.
Epoch 0/99, Loss: 0.1730, Accuracy: 0.7296, Val Accuracy 0.7616, balanced_acc: 0.5794, roc_auc_score: 0.6643 


|2025-01-03|20:11:09.851| [WARNING] /tmp/ipykernel_873295/3560450480.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 82/82 [00:11<00:00,  7.18it/s]

Epoch 1/99, Loss: 0.1264, Accuracy: 0.7846, Val Accuracy 0.7849, balanced_acc: 0.5403, roc_auc_score: 0.7048 


...

|2025-01-03|22:23:57.283| [WARNING] /tmp/ipykernel_873295/3560450480.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 82/82 [00:07<00:00, 10.97it/s]

Epoch 98/99, Loss: 0.0597, Accuracy: 0.9051, Val Accuracy 0.8280, balanced_acc: 0.7388, roc_auc_score: 0.8670 


|2025-01-03|22:24:06.497| [WARNING] /tmp/ipykernel_873295/3560450480.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 82/82 [00:07<00:00, 11.63it/s]

Epoch 99/99, Loss: 0.0526, Accuracy: 0.9155, Val Accuracy 0.8477, balanced_acc: 0.7703, roc_auc_score: 0.8841 
TEST: balanced_acc: 0.7573, roc_auc_score: 0.9029


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▆▄▄▄▄▄▄▄▇▆▅▄▄▄▄▃▅▄▃▃▇▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
val_max_balanced_acc,▁
validation_acc,▃▁▅▅▇▆▇▄▃▂▅▅▇▇▇▅▃▆▅▅▇▆█▆▇▆▇▅▆▇▅▆▆█▇▆█▅▇▇
validation_auc,▃▇▇▇█▇▇▇▆▅▁▄▆▆▇▇█▇▇█▆▇▇▆▇▇▇███▇█▇▇▇▇▇▇█▇
validation_balanced_acc,▁▄▆▇▅█▅▇▆▇▁▄▅▅▆▇▇▆▆▇▆▇▆█▇▇█▇▇▇▇▇▇█▆▇▇█▇█
run,99
test_auc,0.90291
test_balanced_acc,0.75727


In [9]:
trained_model = main(num_epochs=100, arch='EfficientNetV2', strategy='focalloss', learning_rate=0.001)


5-fold-MC-CV, Run:  0
['atypical', 'normal']
TRAINING OVERVIEW:


Number of cells in train:  2604 (70.00 percent)
Number of cells in valid:  558 (15.00 percent)
Number of cells in test:  558 (15.00 percent)


|2025-01-03|22:24:19.503| [WARNING] /tmp/ipykernel_873295/3560450480.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 82/82 [00:06<00:00, 11.90it/s]

New best model found.
Epoch 0/99, Loss: 0.1407, Accuracy: 0.7427, Val Accuracy 0.8405, balanced_acc: 0.6950, roc_auc_score: 0.7766 


|2025-01-03|22:24:27.709| [WARNING] /tmp/ipykernel_873295/3560450480.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 82/82 [00:07<00:00, 11.24it/s]

Epoch 1/99, Loss: 0.1042, Accuracy: 0.8210, Val Accuracy 0.8047, balanced_acc: 0.6127, roc_auc_score: 0.8049 


...

|2025-01-03|23:40:13.780| [WARNING] /tmp/ipykernel_873295/3560450480.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 82/82 [00:07<00:00, 11.26it/s]

Epoch 98/99, Loss: 0.0162, Accuracy: 0.9793, Val Accuracy 0.8620, balanced_acc: 0.7563, roc_auc_score: 0.8556 


|2025-01-03|23:40:22.595| [WARNING] /tmp/ipykernel_873295/3560450480.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 82/82 [00:07<00:00, 10.96it/s]

Epoch 99/99, Loss: 0.0109, Accuracy: 0.9866, Val Accuracy 0.8566, balanced_acc: 0.7804, roc_auc_score: 0.8686 
TEST: balanced_acc: 0.7912, roc_auc_score: 0.8857


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▆▆▅▅▃▃▃▃▄▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▁▂▁▂▂▂▁▁▁▁▁▁▁▁▁
val_max_balanced_acc,▁
validation_acc,▇▄▅▆▇▅▇▁▆█▅▃▇▆▆█▅▆▅▅▆▆▇▇▇▅▆▇▅▅▅▇▇▆▆█▇▆▇▇
validation_auc,▄▅▅▄▁▇█▃▅▅▅▆▄▄▆▅█▆▅▆▆▇▇▇▆▆▆▆▇▆▇▆▆▅▄▅▅█▆▅
validation_balanced_acc,▁▁▁▆▅▇▃▇▇▇▄▂▅▅▆▅▆▆▄▄▆▅▄▆▇▆▆▆▄▅▆▆█▅▄▅▄▇▅▆
run,99
test_auc,0.88573
test_balanced_acc,0.79119
